In [1]:
from dataclasses import asdict, dataclass
import os
import torch as th
import gc
import json
import numpy as np
from datetime import datetime
import gc

from src.model_utils import load_nnsight_model, load_sae
from src.exp_utils import compute_llm_artifacts
from src.cache_utils import batch_sae_cache

from src.configs import *


@dataclass
class CacheConfig:
    llm: LLMConfig
    env: EnvironmentConfig

cfg = CacheConfig(
    llm=GEMMA2_LLM_CFG,
    env=ENV_CFG
)

/home/can/dynamic_representations/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, submodule, hidden_dim, num_heads = load_nnsight_model(cfg)

`torch_dtype` is deprecated! Use `dtype` instead!


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.23it/s]


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNo

In [3]:
print(model)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNo

In [4]:
print(model.layers)

AttributeError: Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layerno

In [5]:
print(model.model.layers)

ModuleList(
  (0-25): 26 x Gemma2DecoderLayer(
    (self_attn): Gemma2Attention(
      (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
      (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
      (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
      (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
    )
    (mlp): Gemma2MLP(
      (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
      (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
      (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
      (act_fn): GELUTanh()
    )
    (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
    (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
    (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
    (post_feedforward_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
  )
)


In [6]:
print(model.model.layers[0].self_attn.source)

                              * @deprecate_kwarg("past_key_value", new_name="past_key_values", version="4.58")
                              0 def forward(
                              1     self,
                              2     hidden_states: torch.Tensor,
                              3     position_embeddings: tuple[torch.Tensor, torch.Tensor],
                              4     attention_mask: Optional[torch.Tensor],
                              5     past_key_values: Optional[Cache] = None,
                              6     cache_position: Optional[torch.LongTensor] = None,
                              7     **kwargs: Unpack[FlashAttentionKwargs],
                              8 ) -> tuple[torch.Tensor, Optional[torch.Tensor], Optional[tuple[torch.Tensor]]]:
                              9     input_shape = hidden_states.shape[:-1]
                             10     hidden_shape = (*input_shape, -1, self.head_dim)
                             11 
 self_q_proj_0         

In [ ]:
with th.inference_mode():
    with model.trace("Hello, I am Can!", scan=False, validate=False, output_attentions=True):
        attn_pattern_BAPP = submodule.self_attn.output[1].save()
        out = submodule.output
        if isinstance(out, tuple):
            out = out[0]
        out.save()
attn_pattern_APP.shape

torch.Size([1, 8, 7, 7])